From https://docs.pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [1]:
import pathlib

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils import data
from torchvision import datasets, transforms

In [2]:
# data loader
BATCH_SIZE = 64
# nn
W1_NUM_NEURONS = 512
W2_NUM_NEURONS = 512
NUM_CLASSES = 10
# optim
LEARNING_RATE = 1e-3
# training
NUM_EPOCHS = 5
# inference
INFERENCE_SAMPLE = 320

In [3]:
# download data
training_data = datasets.FashionMNIST(root="data", train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=transforms.ToTensor())

In [4]:
# create data loaders
train_dataloader = data.DataLoader(training_data, batch_size=BATCH_SIZE)
test_dataloader = data.DataLoader(test_data, batch_size=BATCH_SIZE)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break
X_c, X_h, X_w = X.shape[1:]

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else "cpu"
print(f"Using device: {device}")


class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(X_h * X_w, W1_NUM_NEURONS),
            nn.ReLU(),
            nn.Linear(W1_NUM_NEURONS, W2_NUM_NEURONS),
            nn.ReLU(),
            nn.Linear(W2_NUM_NEURONS, NUM_CLASSES),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)

Using device: mps
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [7]:
def train(
    dataloader: data.DataLoader,
    model: nn.Module,
    loss_fn: torch.nn.modules.loss._Loss,
    optimizer: torch.optim.Optimizer,
) -> None:
    size = len(dataloader.dataset)
    model.train()  # set train mode
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # forward
        logits = model(X)
        loss = loss_fn(logits, y)

        # backprop
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(
    dataloader: data.DataLoader,
    model: nn.Module,
    loss_fn: torch.nn.modules.loss._Loss,
) -> None:
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            logits = model(X)
            test_loss += loss_fn(logits, y).item()
            correct += (logits.argmax(dim=1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


test(test_dataloader, model, loss_fn)


Test Error: 
 Accuracy: 7.2%, Avg loss: 2.311128 



In [8]:
for t in range(NUM_EPOCHS):
    print(f"Epoch {t + 1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

Epoch 1
-------------------------------
loss: 2.323605  [   64/60000]
loss: 2.303433  [ 6464/60000]
loss: 2.293024  [12864/60000]
loss: 2.282516  [19264/60000]
loss: 2.256087  [25664/60000]
loss: 2.247838  [32064/60000]
loss: 2.247619  [38464/60000]
loss: 2.222991  [44864/60000]
loss: 2.228857  [51264/60000]
loss: 2.192101  [57664/60000]
Test Error: 
 Accuracy: 46.6%, Avg loss: 2.185084 

Epoch 2
-------------------------------
loss: 2.203473  [   64/60000]
loss: 2.186337  [ 6464/60000]
loss: 2.140304  [12864/60000]
loss: 2.157993  [19264/60000]
loss: 2.105243  [25664/60000]
loss: 2.068217  [32064/60000]
loss: 2.093076  [38464/60000]
loss: 2.024635  [44864/60000]
loss: 2.039793  [51264/60000]
loss: 1.974824  [57664/60000]
Test Error: 
 Accuracy: 54.1%, Avg loss: 1.960304 

Epoch 3
-------------------------------
loss: 1.992560  [   64/60000]
loss: 1.959886  [ 6464/60000]
loss: 1.851878  [12864/60000]
loss: 1.901737  [19264/60000]
loss: 1.792642  [25664/60000]
loss: 1.745021  [32064/600

In [9]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [10]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[INFERENCE_SAMPLE][0], test_data[INFERENCE_SAMPLE][1]
with torch.no_grad():
    x = x.to(device)
    logits = model(x)
    print(logits)
    predicted, actual = classes[logits[0].argmax(dim=0)], classes[y]
    print(f"Predicted: {predicted}, Actual: {actual}")


tensor([[-0.2057, -1.0491,  0.1899, -0.6688, -0.0718,  0.4110,  0.0527,  0.0991,
          0.7203,  0.3025]], device='mps:0')
Predicted: Bag, Actual: Bag


In [12]:
pathlib.Path("model.pth").unlink(missing_ok=True)